In [ ]:
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from evaluate.lane import LaneEval
import random
import os
%matplotlib inline


In [ ]:
def show_samples(n_samples, images, randomize=False):
    """Shows multiple images from path list"""
    for sample_n in range(n_samples):
        if randomize:
            n = random.randint(0, len(images)-1)
        else:
            n = sample_n
        label = images[n][-32:-6]
        fig = plt.figure(figsize=(30,6))
        ax = plt.subplot(np.ceil(n_samples/3), 3, sample_n+1)
        ax.set_title(label)
        img = plt.imread(images[n])
        ax.imshow(img)
        ax.set_yticklabels([])
        ax.set_xticklabels([])
    plt.tight_layout()
    plt.show()


### Get Data And Create small subset for testing

In [ ]:
small_set = True
### READ FILE
data_path = "train_set/"
json_gt = [json.loads(line) for line in open('train_set/label_data_0531.json')]
if small_set:
    subset = np.random.choice(json_gt, size=15, replace = False)
    json_gt=subset

def get_gt(json_list, index):
    gt = json_list[index]
    gt_lanes = gt['lanes']
    y_samples = gt['h_samples']
    raw_file = gt['raw_file']
    return [raw_file, gt_lanes, y_samples]

### Examine Data

In [ ]:
from sys import getsizeof
print("Total data: ",len(json_gt))
example_data = get_gt(json_gt,2)
print("Path example: ", example_data[0])
print("Lane example: ", example_data[1])
example_img=[]
for i in range(3):
    example_data=get_gt(json_gt, i)
    example_img.append("train_set/"+ example_data[0])
show_samples(3, example_img)

### Preprocessing

In [ ]:
### Crop, as max index for evaluation is 160, no case
##  in looking above
def roi(img,y):
    """Region of Interest"""
    return(img[y:len(img)])

def draw_lanes(img, gt_lanes, y_samples, thickness=25):
    """raws lines from GT data"""
    gt_lanes_vis = [[(x, y) for (x, y) in zip(lane, y_samples) if x >= 0] for lane in gt_lanes]
    img_vis = np.zeros_like(img)
    for lane in gt_lanes_vis:
        cv2.polylines(img_vis, np.int32([lane]), isClosed=False, color=(0,255,0), thickness= thickness)
    return img_vis

def poly_to_points(poly):
    """Takes a second degree polynomial, returns x_coeffs at y_samples"""
    x=[]
    y=[]
    for y_test in range(160,720,10):
        y.append(y_test)
        pred = y*y*poly[0] + y*poly[1] + poly[2]
        if pred >0 and pred <1280:
            x.append(pred)
        else:
            x.append(-2)
    return x, y

def poly_to_line(img, poly):
    """Draws polynomial to image"""
    line=[]
    for y in range(160,720,10):
        x = y*y*poly[0] + y*poly[1] + poly[2]
        if x>0 and x<1280:
            line.append([np.int32(x), np.int32(y)])
    pts = np.asarray(line, np.int32)
    pts.reshape((-1,1,2))
    output=cv2.polylines(img, [pts], False, (255,0,0), 3)
    return output

def _get_coeffs(gt_lanes, y_samples):
    """Returns order 2 polynomials from ground truth points"""
    polys=[]
    gt_lanes_pts = [[(x, y) for (x, y) in zip(lane, y_samples) if x >= 0] for lane in gt_lanes]
    gt_lanes_pts = [[(x, y) for (x, y) in zip(lane, y_samples) if x >= 0] for lane in gt_lanes]
    for lane in gt_lanes_pts:
        lane = np.asarray(lane)
        x = lane[:,0]
        y = lane[:,1]
        polys.append(np.polyfit(y,x, 2))
    return polys
    

def color_mask(img, lower=[0,210,148], upper=[255,255,255]):
    """Returns mask from color image"""
    return cv2.inRange(img, lower, upper)

def gray_mask(img, lower=230, upper=255):
    """Returns mask from gray image"""    
    return cv2.inRange(img, lower, upper)

def hls_tresh(S_channel, low_limit=0, high_limit=255):
    """Takes saturation channel, outputs binary image"""
    s_binary=np.zeros_like(S_channel)
    s_binary[(S_channel>=low_limit) & (S_channel<=high_limit)]=1
    return s_binary

def tresh_channel(img, lower, upper, channel):
    """Returns threshold in channel"""
    channel_of_interest = img[:,:,channel]
    img_binary = np.zeros_like(channel_of_interest)
    img_binary[(channel_of_interest>=upper)&(channel_of_interest<=upper)]
    return img_binary

### Generate GT mask

In [ ]:
test_data = get_gt(json_gt,6)
img = mpimg.imread(data_path +test_data[0])
mpimg.imsave("out_img/GT_mask/frame.jpg",img)
poly_lanes=_get_coeffs(test_data[1], test_data[2])
mask=draw_lanes(img, test_data[1], test_data[2],5)
mpimg.imsave("out_img/GT_mask/lanes_GT.jpg",mask)
for poly in poly_lanes:
    img=poly_to_line(img, poly)
mpimg.imsave("out_img/GT_mask/poly_reversed_GT.jpg",img)
plt.imshow(img)


In [ ]:
comparission = np.copy(lane_mask)
"""
poly = poly_lanes[0]
line=[]
for y in range(160,720,10):
    x = y*y*poly[0] + y*poly[1] + poly[2]
    if x>0 and x<1280:
        line.append([np.int32(x), np.int32(y)])
print(line)
pts = np.asarray(line, np.int32)
print(pts)
pts.reshape((-1,1,2))
print(pts)


cv2.polylines(comparission, [pts], False, (255,0,0), 3)
"""
for poly in poly_lanes:
    comparission=poly_to_line(comparission, poly)
plt.imshow(comparission)

In [ ]:
hls_img= cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
plt.imshow(hls_img)

In [ ]:
h_channel = tresh_channel(hls_img,220,255, 0)
plt.imshow(1024*h_channel)

In [ ]:
new_img=draw_lanes(img, test_data[1], test_data[2])
plt.imshow(new_img)

In [ ]:
img = cv2.blur(img,(5,5))
s_channel = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)[:,:,0]
s_channel = hls_tresh(s_channel, 100,255)
plt.imshow(s_channel)


In [ ]:
"""
#Open in batch and save subset
for i in range(len(raw_files)):
    img = mpimg.imread(raw_files[i])
    mpimg.imsave("out_img/"+str(i)+".jpg",img)
"""    

"""
for i in range(len(raw_files)):
    img = cv2.imread(raw_files[i])
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = roi(gray,150)
    mpimg.imsave("out_img/"+str(i)+"_crop.jpg",cv2.cvtColor(img, cv2.COLOR_GRAY2RGB))
    img = gray_mask(img, 170,255)
    mpimg.imsave("out_img/"+str(i)+"_mask.jpg",cv2.cvtColor(img, cv2.COLOR_GRAY2RGB))
"""

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)